# Problem Set 5 - Solutions 

In [1]:
import pandas as pd
import numpy as np
from sympy import *

# mysoln is a dictionary to store your answers

# the values of the first key, i.e. "Students" contains names for the students in your group
# make sure these match the names shown on the pdf document provided by the MFE office

mysoln = {'Students':  ["Student A","Student B","Student C", "Student D"] }

# Question 1

In [2]:
# Part 1(a)
A = np.array([
    [100,0,0],[0,1,3],[0,101,103]
])

b = np.array([
    [4000000], 
    [0], 
    [2000000]
]
)

In [3]:
X = np.linalg.solve(A, b)
X

array([[ 40000.],
       [ 30000.],
       [-10000.]])

In [4]:
# Part 1(b)

In [5]:
ytm = 0.02
liabPV = 2000000/(1+ytm)**30 + 4000000/(1+ytm)**31
liabMacD = (2000000/(1+ytm)**30)/liabPV*30 + (4000000/(1+ytm)**31)/liabPV*31
liabModD = liabMacD/(1+ytm)
print(liabPV,liabModD)

bondDp = 100/(1+ytm)**10 
bondEp = 100/(1+ytm)**20
bondDModD = 10/(1+ytm)
bondEModD = 20/(1+ytm)
print(bondDp,bondEp,bondDModD,bondEModD)

3269125.6563124177 30.06103103493053
82.03482998751551 67.29713331080575 9.803921568627452 19.607843137254903


In [6]:
A = np.array([
    [bondDp,bondEp],
    [bondDp*bondDModD, bondEp*bondEModD]
]
)

b = np.array([
    [liabPV],
     [liabPV*liabModD]    
])

In [7]:
X2 = np.linalg.solve(A, b)
X2

array([[-42489.56260016],
       [100372.02727884]])

In [8]:

#X=[1,2,3] #answer for part 1(a)
#X2 = [1,2] # answer for part (1b)
mysolnQ1 = {'a' : X ,'b': X2}
mysoln['Q1'] = mysolnQ1

In [9]:
mysoln

{'Students': ['Student A', 'Student B', 'Student C', 'Student D'],
 'Q1': {'a': array([[ 40000.],
         [ 30000.],
         [-10000.]]),
  'b': array([[-42489.56260016],
         [100372.02727884]])}}

# Question 2

In [10]:
# a 
def convex(cpercent, maturity, yield1, face, py = 2):
    c = cpercent/py*face #coupon
    yieldsa = yield1/py #semi-annual yield
    totalsteps = int(maturity*py)
    periods = np.linspace(1/py,stop = maturity,num = totalsteps) #num periods
    cf = np.repeat(c,len(periods)) +np.append(np.repeat(0,len(periods)-1),100)
    price = sum(cf*(1+yieldsa)**(-py*periods))
    gamma = sum(periods*(periods+1)*cf*(1+yieldsa)**(-py*periods))/(1+yieldsa)**2/price
    return gamma
    


In [11]:
# bond price helper function
def bondprice(cpercent, maturity, yield1, face,py):
    c = cpercent/py * face #coupon
    yieldsa = yield1/py #semi-annual yield
    totalsteps = int(maturity*py)
    periods = np.linspace(1/py,stop = maturity,num = totalsteps) #num periods
    price = np.cumsum(c*(1+yieldsa)**(-py*periods))
    price = price[maturity*py-1] + face*(1+yieldsa)**(-py*maturity)
    return price
    
    

In [12]:
#modified duration helper function
def modd(cpercent, maturity, yield1, face, py=2):
    c = cpercent/py * face #coupon
    yieldsa = yield1/py #semi-annual yield
    totalsteps = int(maturity*py)
    periods = np.linspace(1/py,stop = maturity,num = totalsteps) #num periods
    cf = np.repeat(c,len(periods)) +np.append(np.repeat(0,len(periods)-1),100)
    weightedprice = sum(periods*cf*(1+yieldsa)**(-py*periods)) #discounted weighted cash flows at each period
    price = sum(cf*(1+yieldsa)**(-py*periods)) #discounted cash flows at each period
    return weightedprice/price/(1+yield1/py)

In [13]:
def convexfd(cpercent, maturity, yield1, face, py=2, h=0.000001):
    price = bondprice(cpercent, maturity, yield1, face, py=py)
    priceL = bondprice(cpercent, maturity, yield1-h, face, py=py)
    priceH = bondprice(cpercent, maturity, yield1+h, face, py=py)
    return((priceH+priceL - 2*price)/(h**2) / price)
    

In [14]:
yield1=0.02
coupon=0.02
face=100
#the price calculations are unnecessary if we recognize that the yield
#and coupon payments are the same, so the bond must be issued at par.
#we use our functions from question 1
bond1price = bondprice(coupon, 1, yield1, face, py=1)
bond1duration = modd(coupon, 1, yield1, face, py=1)
bond1convexity = convex(coupon, 1, yield1, face, py=1)
bond1convexitychk = convexfd(coupon, 1, yield1, face, py=1)
a = [bond1price, bond1duration, bond1convexity]


In [15]:
a

[99.99999999999999, 0.9803921568627451, 1.9223375624759709]

In [16]:
bond2price = bondprice(coupon, 2, yield1, face, py=1)
bond2duration = modd(coupon, 2, yield1, face, py=1)
bond2convexity = convex(coupon, 2, yield1, face, py=1)
b = [bond2price, bond2duration, bond2convexity]

In [17]:
b

[99.99999999999999, 1.9415609381007308, 5.691626900664149]

In [18]:
#no difference in expected cashflows for the frn 
#as compared with the two year bond
yield1 = 0.02
frnprice = bondprice(coupon, 2, yield1, face, py=1)
c = [frnprice]
c = np.round(c)

In [19]:
frnduration = 1/(1+yield1)
frnconvexity= 2/(1+yield1)**2
d = [frnduration, frnconvexity]
d


[0.9803921568627451, 1.9223375624759709]

In [20]:
mysolnQ2 = {'a' : a ,'b': b, 'c':c, 'd':d}
mysoln['Q2'] = mysolnQ2
mysoln

{'Students': ['Student A', 'Student B', 'Student C', 'Student D'],
 'Q1': {'a': array([[ 40000.],
         [ 30000.],
         [-10000.]]),
  'b': array([[-42489.56260016],
         [100372.02727884]])},
 'Q2': {'a': [99.99999999999999, 0.9803921568627451, 1.9223375624759709],
  'b': [99.99999999999999, 1.9415609381007308, 5.691626900664149],
  'c': array([100.]),
  'd': [0.9803921568627451, 1.9223375624759709]}}